In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

In [ ]:
INPUT_DIR_A = '../data/train_a'
INPUT_DIR_TEST = '../data/test_4_a'

In [ ]:
# Define the batch size
batch_size = 1

# Define the transformation
transform = transforms.Compose([
    transforms.RandomPerspective(p=1),  # Apply a random perspective transformation
    transforms.ToTensor(),  # Convert the image to a tensor
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

# Define the dataset
dataset = datasets.ImageFolder(INPUT_DIR_A, transform=transform)

# Verify the classes
classes = dataset.classes
print(classes)

dataset = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Device: {device}")

# Load the pre-trained ResNet model
model = resnet50(weights='ResNet50_Weights.DEFAULT')

# Freeze all the parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes in your dataset
num_classes = len(classes)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.fc.parameters(), lr=0.1)

# Define the number of training epochs
num_epochs = 100

# Training loop
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Set the model to training mode
    model.train()

    
    for images, labels in dataset:
        # Move the images and labels to the GPU if available
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update the running loss
        running_loss += loss.item()
        
        # Compute the predicted labels
        _, predicted = torch.max(outputs.data, 1)
        
        # Update the total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Compute the accuracy
    accuracy = 100 * correct / total
    
    # Print the loss and accuracy for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}: Loss = {running_loss:.4f}, Accuracy = {accuracy:.2f}%")
    if accuracy > 90:
        break


In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder

# Define transformations for the test data
test_transforms = transforms.Compose([
    transforms.ToTensor(),
])

# Load the test dataset
#test_dataset = ImageFolder(INPUT_DIR_TEST, transform=test_transforms)
test_dataset = ImageFolder(INPUT_DIR_TEST, transform=test_transforms)
test_classes = test_dataset.classes

# Create a DataLoader for the test dataset
test_dataloader = DataLoader(test_dataset, batch_size=1)


In [ ]:

model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Turn off gradients for prediction, saves memory and computations
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        softmax_output = F.softmax(outputs, dim=1).cpu().numpy()[:, predicted]
        print(f'Actual: {test_classes[labels]}, Predicted: {classes[predicted]} {(softmax_output[0]*100):.2f}%')

